# Read or Write GAMMA files

**Copyright (c) 2021, Lei Yuan**

## all functions

In [1]:
from pykml.factory import KML_ElementMaker as KML
import matplotlib.pyplot as plt
import matplotlib as mpl
from lxml import etree
import numpy as np
import os
import sys
import zipfile
from osgeo import gdal, osr


def read_gamma(file, lines, file_type):
    # check file
    if not os.path.isfile(file):
        sys.exit('{} does not exist.'.format(file))
    data = np.fromfile(file, dtype=file_type)
    # GAMMA output files are big-endian
    data.byteswap('True')
    data = data.reshape(lines, -1)

    return data


def write_gamma(data, out_file, file_type):
    data = data.astype(file_type)
    data.byteswap('True')
    data.reshape(-1, 1)
    print('Writing data to {}'.format(out_file))
    data.tofile(out_file)
    print('done.')


def view_data(data, vlim=None, cmap='jet', title=None, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    if title:
        ax.set_title(title)
    ax.axis('off')

    if vlim:
        minv, maxv = vlim
    else:
        minv, maxv = np.nanmin(data), np.nanmax(data)

    im = ax.imshow(data, cmap=cmap, vmin=minv, vmax=maxv)

    fig.colorbar(im, ax=ax, pad=0.02, ticks=[minv, 0, maxv])


def wrap_data(data, wrap_range=[-np.pi, np.pi]):
    w0, w1 = wrap_range
    wrapped_data = w0 + np.mod(data - w0, w1 - w0)

    return wrapped_data


def read_dem_par(par_file):
    par_key = [
        'width', 'nlines', 'corner_lat', 'corner_lon', 'post_lat', 'post_lon'
    ]
    par_value = []
    with open(par_file, 'r') as f:
        for line in f.readlines():
            for i in par_key:
                if line.strip().startswith(i):
                    value = line.strip().split()[1]
                    if i in ['width', 'nlines']:
                        par_value.append(int(value))
                    else:
                        par_value.append(float(value))

    return par_value


def get_lon_lat(dem_par):
    par_value = read_dem_par(dem_par)
    north = par_value[2]
    south = north + par_value[4] * par_value[1]
    west = par_value[3]
    east = west + par_value[5] * par_value[0]

    return [south, north, west, east]


def data2xyz(data, dem_par, out_file):
    par_value = read_dem_par(dem_par)
    width, length = par_value[0], par_value[1]
    upper_left_lat, upper_left_lon = par_value[2], par_value[3]
    lat_step, lon_step = par_value[4], par_value[5]

    lon = np.linspace(upper_left_lon, upper_left_lon + lon_step * width, width)
    lat = np.linspace(upper_left_lat, upper_left_lat + lat_step * length, length)
    lons, lats = np.meshgrid(lon, lat)

    lons = lons.reshape((-1, 1))
    lats = lats.reshape((-1, 1))
    data = data.reshape((-1, 1))

    not_nan = ~np.isnan(data)
    data = data[not_nan].reshape((-1, 1))
    lons = lons[not_nan].reshape((-1, 1))
    lats = lats[not_nan].reshape((-1, 1))

    out_data = np.hstack((lons, lats, data))
    print('Writing data to {}'.format(out_file))
    np.savetxt(out_file, out_data, fmt='%4f')
    print('done.')


def data2geotiff(data, dem_par, outfile):
    par_value = read_dem_par(dem_par)
    width, nlines = par_value[0], par_value[1]
    lat_step, lon_step = par_value[4], par_value[5]
    lon_lat = get_lon_lat(dem_par)
    max_lat, min_lon = lon_lat[1], lon_lat[2]

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(outfile, width, nlines, 1, gdal.GDT_Float32)

    dataset.SetGeoTransform([min_lon, lon_step, 0, max_lat, 0, lat_step])

    sr = osr.SpatialReference()
    sr.SetWellKnownGeogCS('WGS84')

    dataset.SetProjection(sr.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(data)
    print('Writing data to {}.'.format(outfile))
    dataset.FlushCache()
    dataset = None
    print('done.')


def draw_colorbar(vlim, cmap, label, out_file, figsize=(0.18, 3.6)):
    fig, ax = plt.subplots(figsize=figsize)

    cmap = plt.get_cmap(cmap)
    minv, maxv = vlim
    norm = mpl.colors.Normalize(vmin=minv, vmax=maxv)
    fig.patch.set_facecolor('white')
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='vertical', label=label, ticks=[minv, 0, maxv])
    fig.savefig(out_file, bbox_inches='tight', dpi=300)


def draw_img_pwr(img_data, cmap, pwr_data=None, vlim=None, out_file=None, withcmap=False, figsize=(10, 10)):
    if vlim:
        minv, maxv = vlim
    else:
        minv, maxv = np.nanmin(img_data), np.nanmax(img_data)

    img_data[img_data==0.0] = np.nan
    # scale data to the range 0 -> 1
    dt = (img_data - minv) / (maxv - minv)

    # create RGB display of the data in the range 0 -> 1
    cmap = mpl.cm.get_cmap(cmap)
    img_rgba = cmap(dt, bytes=True)
    r, g, b, a = np.rollaxis(img_rgba, axis=-1)

    if pwr_data is not None:
        exp, sc = 0.35, 1.0
        pwr = np.abs(pwr_data)
        # all points that are non-zero
        idx = np.nonzero(pwr)
        # scale these data values and evaluate the mean value
        pwr = np.power(pwr, exp)  
        ave = pwr[idx].mean()
        # determine default clip value
        cv = 2.0 * ave / sc
        v = (pwr.clip(0., cv)) / cv
        v[v==0.0] = np.nan
        # multiply by v to get scaled RGB
        r = (r.astype(np.float32) * v).astype(np.uint8)
        b = (b.astype(np.float32) * v).astype(np.uint8)
        g = (g.astype(np.float32) * v).astype(np.uint8)

    img_rgba = np.dstack([r, g, b, a])

    fig, ax = plt.subplots(figsize=figsize)
    ax.axis('off')

    if pwr_data is None:
        ax.imshow(img_rgba)
    else:
        ax.imshow(v, zorder=0, cmap='gray')
        ax.imshow(img_rgba, zorder=1)

    if withcmap:
        norm = mpl.colors.Normalize(vmin=minv, vmax=maxv)
        fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax, orientation='vertical', pad=0.02, ticks=[minv, 0, maxv])

    if out_file:
        fig.savefig(out_file, pad_inches=0.0, bbox_inches='tight', dpi=600, transparent=True)


def make_kmz(data, lon_lat, out_kmz, pwr_data=None, cmap='jet', vlim=None, label=None):
    out_kmz = os.path.abspath(out_kmz)
    if out_kmz.endswith('.kmz'):
        out_kmz = out_kmz[:-4]

    out_dir = os.path.dirname(out_kmz)
    cbar_file = os.path.join(out_dir, 'cbar.png')
    img_file = os.path.join(out_dir, 'img.png')

    if vlim is None:
        vlim = [np.nanmin(data), np.nanmax(data)]

    print('drawing colorbar')
    draw_colorbar(vlim, cmap, label, cbar_file)
    if pwr_data:
        print('drawing image with pwr data')
    print('drawing image')
    draw_img_pwr(data, cmap, pwr_data=pwr_data, vlim=vlim, out_file=img_file, withcmap=False)

    south, north, west, east = lon_lat

    doc = KML.kml(KML.Folder(KML.name(os.path.basename(out_kmz))))
    img_displayed = KML.GroundOverlay(
        KML.name('img'), KML.Icon(KML.href('img.png')),
        KML.LatLonBox(KML.north(str(north)), KML.south(str(south)), KML.east(str(east)), KML.west(str(west))))
    doc.Folder.append(img_displayed)

    legend = KML.ScreenOverlay(
        KML.name('colorbar'),
        KML.Icon(KML.href('cbar.png'), KML.viewBoundScale(0.75)),
        KML.overlayXY(
            x="0.0",
            y="1",
            xunits="fraction",
            yunits="fraction",
        ), KML.screenXY(
            x="0.0",
            y="1",
            xunits="fraction",
            yunits="fraction",
        ),
        KML.rotationXY(
            x="0.",
            y="1.",
            xunits="fraction",
            yunits="fraction",
        ), KML.size(
            x="0",
            y="250",
            xunits="pixel",
            yunits="pixel",
        ), KML.visibility(1), KML.open(0))

    doc.Folder.append(legend)

    kml_str = etree.tostring(doc, pretty_print=True)
    kml = out_kmz + '.kml'
    with open(kml, 'wb') as f:
        f.write(kml_str)

    kmz = out_kmz + '.kmz'
    print('writing data to {}'.format(kmz))
    with zipfile.ZipFile(kmz, 'w') as f:
        os.chdir(out_dir)
        f.write(os.path.basename(kml))
        os.remove(kml)
        f.write('img.png')
        os.remove('img.png')
        f.write('cbar.png')
        os.remove('cbar.png')


def cm2cpt(cm_file, cpt):
    cm = np.loadtxt(cm_file, dtype=np.uint8)
    lines = cm.shape[0]
    step = np.linspace(0,1,lines)
    with open(cpt, 'w+', encoding='utf-8') as f:
        bar = '-' * 85
        f.write(f'#{bar}\n')
        f.write('# COLOR_MODEL = RGB\n')
        f.write(f'#{bar}\n')
        for l in range(lines-1):
            rgb0 = f'{cm[l][0]}/{cm[l][1]}/{cm[l][2]}'
            rgb1 = f'{cm[l+1][0]}/{cm[l+1][1]}/{cm[l+1][2]}'

            f.write(f'{step[l]:<22} {rgb0:<13} {step[l+1]:<22} {rgb1:<13}\n')

        f.write(f'#{bar}\n')
        f.write(f'B {cm[0][0]}/{cm[0][1]}/{cm[0][2]}\n')
        f.write(f'F white\n')
        f.write(f'N {cm[0][0]}/{cm[0][1]}/{cm[0][2]}\n')
        f.write(f'#{bar}\n')


def register_cmap(cmap_file, cmap_name):
    cm = mpl.colors.ListedColormap(np.loadtxt(cmap_file, dtype='float') / 255.0, cmap_name)
    plt.cm.register_cmap(cmap=cm)

    return cm

cmap_dir = os.path.join(os.environ['TINTPY_HOME'], 'tintpy', 'data', 'colormaps')
if os.path.isdir(cmap_dir):
    cm = register_cmap(os.path.join(cmap_dir, 'rmg_hs.cm'), 'rmg_hs')

## read unwrapped file

In [ ]:
os.chdir(r'/media/ly/file/Iran/DInSAR/diff')

unw = read_gamma('20171111_20171117.adf.unw', 1362, np.float32)

view_data(unw, cmap='rmg_hs', title='unw', figsize=(8, 8), vlim=[-201, 201])

# write_gamma(unw, 'file_by_python.unw', np.float32)

## read coherence

In [ ]:
cc = read_gamma('20171111_20171117.adf.cc', 1362, 'float32')
cc[cc==0]=np.nan
view_data(cc, cmap='gray', title='cc')

# write_gamma(cc, 'file_by_python.cc', 'float32)

## read interferogram

In [ ]:
data = read_gamma('20171111_20171117.adf.diff', 1362, 'complex64')

# rmg_hs is recommanded for interferogram
view_data(np.angle(data), cmap='rmg_hs', title='diff')

# write_gamma(data, 'file_by_python.diff', 'complex64)

## wrap data

In [ ]:
data = read_gamma('20171111_20171117.adf.unw', 1362, np.float32)

wrapped_data = wrap_data(data, wrap_range=[-np.pi, np.pi])

# rmg_hs is recommanded for interferogram
view_data(wrapped_data, cmap='rmg_hs', title='wrap')

## mask data using coherence

In [ ]:
unw = read_gamma('20171111_20171117.adf.unw', 1362, np.float32)
cc = read_gamma('20171111_20171117.adf.cc', 1362, np.float32)

unw[cc < 0.2] = np.nan

view_data(unw, cmap='jet', title='masked unw')

## data2xyz

In [ ]:
unw = read_gamma('20171111_20171117.adf.unw', 1362, np.float32)

data2xyz(unw, 'dem_seg.par', 'unw.txt')

## data2geotiff

In [ ]:
unw = read_gamma('20171111_20171117.adf.unw', 1362, np.float32)

data2geotiff(unw, 'dem_seg.par', 'unw.tif')

## data2kmz

In [ ]:
os.chdir('/media/ly/file/Iran/DInSAR/geo')

lon_lat = get_lon_lat('dem_seg.par')
_, nlines, _, _, _, _ = read_dem_par('dem_seg.par')

unw = read_gamma('20171111_20171117.adf.unw.geo', nlines, 'float32')

make_kmz(unw, lon_lat, 'unw_no_pwr.kmz', pwr_data=None, cmap='rmg_hs', vlim=[-201, 201], label='phase')

In [ ]:
# using pwr as background image

lon_lat = get_lon_lat('dem_seg.par')
_, nlines, _, _, _, _ = read_dem_par('dem_seg.par')

unw = read_gamma('20171111_20171117.adf.unw.geo', nlines, 'float32')

pwr_data = read_gamma('20171111.rmli.geo', nlines, 'float32')

make_kmz(unw, lon_lat, 'unw_pwr.kmz', pwr_data=pwr_data, cmap='rmg_hs', vlim=[-201, 201], label='phase')